In [1]:
from pydantic import BaseModel, Field

import openai
import instructor
from qdrant_client import QdrantClient

### Mock an Example

In [2]:
prompt = """
You are a helpful assistant.
Return an answer to the question.
Question: What is your name?
"""

In [3]:
openai_response = openai.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": prompt}
    ],
    temperature=0
)

print(openai_response.choices[0].message.content)

I am ChatGPT, your AI assistant. How can I help you today?


### Add structured outputs (Instructor)

In [4]:
client = instructor.from_openai(openai.OpenAI())

In [7]:
class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="The answer to the question")

In [10]:
response, raw_response= client.chat.completions.create_with_completion(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": prompt}
    ],
    temperature=0,
    response_model= RAGGenerationResponse
)


In [14]:
response

RAGGenerationResponse(answer='My name is ChatGPT. How can I assist you today?')

In [12]:
raw_response

ChatCompletion(id='chatcmpl-D04yEOxnuQFOjuNDHp4Z17DTbkQW9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_9qKSHHr33FcSVYK5ZkQ3Gohc', function=Function(arguments='{"answer":"My name is ChatGPT. How can I assist you today?"}', name='RAGGenerationResponse'), type='function')]))], created=1768911478, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_376a7ccef1', usage=CompletionUsage(completion_tokens=17, prompt_tokens=95, total_tokens=112, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=None), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [15]:
class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="The answer to the question")
    reasoning: str = Field(description="The reasoning for the answer")

In [16]:

response, raw_response = client.chat.completions.create_with_completion(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": prompt}
    ],
    temperature=0,
    response_model=RAGGenerationResponse
)

In [17]:
response

RAGGenerationResponse(answer='My name is ChatGPT.', reasoning='I am an AI language model created by OpenAI, and I am commonly known as ChatGPT.')

## RAG Example

In [26]:
class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="The answer to the question")

In [27]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }


def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt


def generate_answer(prompt):

    response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0,
        response_model=RAGGenerationResponse
    )

    return response, raw_response


def rag_pipeline(question, qdrant_client, top_k=5):

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer, raw_response = generate_answer(prompt)

    return {
        "datamodel": answer,
        "answer": answer.answer,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"]
    }

In [28]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [29]:
output = rag_pipeline("Can I get a charging cable? Please suggest me a good one.", qdrant_client)

In [30]:
output

{'datamodel': RAGGenerationResponse(answer='Based on the available products, I suggest the GREPHONE 2 Pack USB C to Lightning Cable, 6 FT MFi Certified (ID: B0BV6PWVCG). It offers fast charging, is Apple MFi certified for compatibility and safety, and has a durable design with reinforced material and aluminum joints. The 6-foot length is convenient for use while charging. It supports fast charging up to 3A/30W and is compatible with a wide range of iPhone and iPad models.\n\nIf you prefer a multi-device charging cable, the 5 in 1 USB C to Multi Charging Cable 3M/10Ft (ID: B0BFPZGYLD) is also a good option. It can charge three devices simultaneously with multiple connectors (Lightning, Type C, Micro USB) and is Apple MFi certified.\n\nFor a standard iPhone charging cable pack, the iPhone Charger Cord Lightning Cables 3Pack 3ft (ID: B0BYYLJRHT) is highly rated and Apple MFi certified, offering durability and fast charging.\n\nLet me know if you want details on any of these options!'),
 '

In [31]:
print(output['answer'])

Based on the available products, I suggest the GREPHONE 2 Pack USB C to Lightning Cable, 6 FT MFi Certified (ID: B0BV6PWVCG). It offers fast charging, is Apple MFi certified for compatibility and safety, and has a durable design with reinforced material and aluminum joints. The 6-foot length is convenient for use while charging. It supports fast charging up to 3A/30W and is compatible with a wide range of iPhone and iPad models.

If you prefer a multi-device charging cable, the 5 in 1 USB C to Multi Charging Cable 3M/10Ft (ID: B0BFPZGYLD) is also a good option. It can charge three devices simultaneously with multiple connectors (Lightning, Type C, Micro USB) and is Apple MFi certified.

For a standard iPhone charging cable pack, the iPhone Charger Cord Lightning Cables 3Pack 3ft (ID: B0BYYLJRHT) is highly rated and Apple MFi certified, offering durability and fast charging.

Let me know if you want details on any of these options!
